In [1]:
%pip install -Uq chromadb datasets
%pip uninstall -y thinc spacy
%pip install numpy==1.26.4 datasets --upgrade

Note: you may need to restart the kernel to use updated packages.


'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.3 requires chromadb!=0.5.4,!=0.5.5,<0.6.0,>=0.4.0, but you have chromadb 1.0.20 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


Note: you may need to restart the kernel to use updated packages.


'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
# Get the SciQ dataset from HuggingFace
from datasets import load_dataset
dataset = load_dataset("sciq", split="train")
print("Number of questions : ", len(dataset))

c:\Users\aaab9\miniconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aaab9\miniconda3\envs\langchain_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aaab9\.cache\huggingface\hub\datasets--sciq. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate de

Number of questions :  11679


In [3]:
type(dataset)

datasets.arrow_dataset.Dataset

In [4]:
dataset[0]

{'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'distractor3': 'viruses',
 'distractor1': 'protozoa',
 'distractor2': 'gymnosperms',
 'correct_answer': 'mesophilic organisms',
 'support': 'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}

In [5]:
dataset = dataset.filter(lambda x: x["support"] != "")
print("Number of questions with support: ", len(dataset))

Filter: 100%|██████████| 11679/11679 [00:00<00:00, 21850.33 examples/s]

Number of questions with support:  10481


In [9]:
dataset['question'][:10]

['What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?',
 'Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?',
 'What is the least dangerous radioactive decay?',
 'Kilauea in hawaii is the world’s most continuously active volcano. very active volcanoes characteristically eject red-hot rocks and lava rather than this?',
 'When a meteoroid reaches earth, what is the remaining object called?',
 'What kind of a reaction occurs when a substance reacts quickly with oxygen?',
 'Organisms categorized by what species descriptor demonstrate a version of allopatric speciation and have limited regions of overlap with one another, but where they overlap they interbreed successfully?.',
 'Alpha emission is a type of what?',
 'What is

In [10]:
dataset["correct_answer"][10]

'active metal'

In [11]:
dataset['support'][10]

'One way to keep iron from corroding is to keep it painted. The layer of paint prevents the water and oxygen necessary for rust formation from coming into contact with the iron. As long as the paint remains intact, the iron is protected from corrosion. Other strategies include alloying the iron with other metals. For example, stainless steel is mostly iron with a bit of chromium. The chromium tends to collect near the surface, where it forms an oxide layer that protects the iron. Zinc-plated or galvanized iron uses a different strategy. Zinc is more easily oxidized than iron because zinc has a lower reduction potential. Since zinc has a lower reduction potential, it is a more active metal. Thus, even if the zinc coating is scratched, the zinc will still oxidize before the iron. This suggests that this approach should work with other active metals. Another important way to protect metal is to make it the cathode in a galvanic cell. This is cathodic protection and can be used for metals 

In [12]:
# Import Chroma and instantiate a client.
# The default Chroma client is ephemeral, meaning it will not save to disk.
# Chroma를 가져와서 클라이언트를 인스턴스화합니다.
# 기본 Chroma 클라이언트는 임시적이므로 디스크에 저장되지 않습니다.

import chromadb
client = chromadb.Client()

In [14]:
# Create a new Chroma collection to store the supporting evidence.
# We don't need to specify an embedding fuction, and the default will be used.
# 새 크로마 컬렉션을 만들어 증빙 자료를 저장합니다.
# 임베딩 함수를 지정할 필요는 없으며 기본값이 사용됩니다.
collection = client.create_collection("sciq_supports")

In [15]:
type(collection)

chromadb.api.models.Collection.Collection

In [16]:
dataset

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 10481
})

In [18]:
len(dataset['support'][14:])

10467

In [19]:
for idx,text in enumerate(dataset['support'][14:20],start=15): # 14 ~ 19
    print(f"{idx}번째 support : {text}")

15번째 support : Gene transcription is controlled by regulatory proteins that bind to regulatory elements on DNA. The proteins usually either activate or repress transcription.
16번째 support : Occurrence, Preparation, and Compounds of Boron and Silicon Boron constitutes less than 0.001% by weight of the earth’s crust. In nature, it only occurs in compounds with oxygen. Boron is widely distributed in volcanic regions as boric acid, B(OH)3, and in dry lake regions, including the desert areas of California, as borates and salts of boron oxyacids, such as borax, Na2B4O7⋅10H2O. Elemental boron is chemically inert at room temperature, reacting with only fluorine and oxygen to form boron trifluoride, BF3, and boric oxide, B2O3, respectively. At higher temperatures, boron reacts with all nonmetals, except tellurium and the noble gases, and with nearly all metals; it oxidizes to B2O3 when heated with concentrated nitric or sulfuric acid. Boron does not react with nonoxidizing acids. Many boron com

In [20]:
len(list(zip(dataset['question'],dataset['correct_answer'],dataset['support']))[:100])

100

In [21]:
data_pair = list(zip(dataset['question'],dataset['correct_answer'],dataset['support']))[:100]

num = 20

print(
    f" 1. question : {data_pair[num][0]}\n",
    f"2. correct_answer : {data_pair[num][1]}\n",
    f"3. support : {data_pair[num][2]}\n",
)

 1. question : Most of the chemical reactions in the body are facilitated by what?
 2. correct_answer : enzymes
 3. support : Enzymes are critical to the body’s healthy functioning. They assist, for example, with the breakdown of food and its conversion to energy. In fact, most of the chemical reactions in the body are facilitated by enzymes.



In [23]:
collection

Collection(name=sciq_supports)

In [22]:
# Embed and store the first 100 supports for this demo
# 이 데모에 대한 앞 100개의 support 임베딩 및 저장하기
collection.add(
    ids=[str(i) for i in range(0, 100)],  # IDs are just strings
    documents=dataset["support"][:100],
    metadatas=[{"type": "support"} for _ in range(0, 100)],
)

C:\Users\aaab9\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:14<00:00, 618kiB/s] 


In [24]:
results = collection.query(
    query_texts=dataset["question"][:10],
    n_results=5)

In [25]:
results

{'ids': [['36', '9', '30', '0', '89'],
  ['1', '34', '39', '87', '88'],
  ['2', '80', '41', '84', '65'],
  ['3', '8', '13', '66', '51'],
  ['4', '42', '39', '34', '3'],
  ['5', '89', '4', '58', '83'],
  ['6', '46', '45', '51', '67'],
  ['7', '68', '19', '66', '92'],
  ['8', '3', '57', '32', '13'],
  ['9', '29', '19', '83', '18']],
 'embeddings': None,
 'documents': [['Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.',
   'The stored fo

In [28]:
results.keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances'])

In [31]:
list(zip(results['distances'][0],results['documents'][0]))

[(1.1048438549041748,
  'Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.'),
 (1.2558344602584839,
  'The stored food in a seed is called endosperm . It nourishes the embryo until it can start making food on its own.'),
 (1.3100907802581787,
  'Photosynthetic protists use light energy to make food. They are major producers in aquatic ecosystems.'),
 (1.3559309244155884,
  'Mesophiles grow best in moderate temperature, typically between

In [33]:
combined=[(results['distances'][i][0],results['ids'][i][0]) for i in range(0,10)]
combined

[(1.1048438549041748, '36'),
 (0.46667468547821045, '1'),
 (0.9318017959594727, '2'),
 (0.418067991733551, '3'),
 (1.0375895500183105, '4'),
 (0.5699304342269897, '5'),
 (0.791527509689331, '6'),
 (0.7859898805618286, '7'),
 (0.6113287210464478, '8'),
 (0.4140006899833679, '9')]

In [35]:
(results['distances'][3][0],results['ids'][3][0])

(0.418067991733551, '3')

In [36]:
# Print the question and the corresponding support
for i,(q,s) in enumerate(zip(dataset['question'][:10], dataset['support'][:10])):
    print(f"Question: {q}")
    print(f"correct support :{s}")
    print(f"Retrieved support: {results['documents'][i][0]}")
    print()

Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
correct support :Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.
Retrieved support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without s